In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Reasoning Engine in Vertex AI - Chat history management

* This colab explains how to deal with chat history being stored in firestore.   
* Firestore reference : https://python.langchain.com/v0.2/docs/integrations/memory/google_firestore/

## Init Vertex AI SDK

In [1]:
%pip install --upgrade --quiet --user \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    "google-cloud-firestore" \
    "langchain-google-firestore" \
    cloudpickle==3.0.0 \
    pydantic==2.7.4 \
    requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.0/409.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00


In [2]:
# @title Authentication to access to GCP

# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Configuration and initialization

In [3]:
# @title Define constants
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-1.5-flash-002" # @param {type:"string"}

In [8]:
# @title Firestore configuration
DATABASE="chat-history" # @param {type:"string"}
COLLECTION="history"    # @param {type:"string"}

In [4]:
# @title Create a bucket.
BUCKET_URI = f"gs://mlops-{PROJECT_ID}-1209"
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://mlops-ai-hangsik-1209/...
ServiceException: 409 A Cloud Storage bucket named 'mlops-ai-hangsik-1209' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [5]:
# @title Service account
shell_output = ! gcloud projects describe  $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")

SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

print(f"SERVICE_ACCOUNT: {SERVICE_ACCOUNT}")

SERVICE_ACCOUNT: 721521243942-compute@developer.gserviceaccount.com


In [6]:
# @title Set access to the bucket
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

No changes made to gs://mlops-ai-hangsik-1209/
No changes made to gs://mlops-ai-hangsik-1209/


In [7]:
# @title Initialize Vertex AI with Staging Bucket.

import vertexai
from vertexai.preview import reasoning_engines

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

## Firestore setting for chat history

Before executing the following cell, you have to set up firestore database in GCP Console.
* Create database : https://cloud.google.com/firestore/docs/manage-databases

### Helper method to get session history

In [10]:
# refer to : https://python.langchain.com/v0.2/docs/integrations/memory/google_firestore/#custom-client

def get_session_history(session_id: str):
    """
    Retrieve chat history data with session id for each conversation from Firestore on GCP
    """

    from langchain_google_firestore import FirestoreChatMessageHistory
    from google.cloud import firestore

    client = firestore.Client(project=PROJECT_ID, database=DATABASE)

    chat_history = FirestoreChatMessageHistory(
                      client=client,
                      session_id=session_id,
                      collection=COLLECTION,
                      encode_message=False,
    )

    return chat_history

def get_converstion(chat_history):
  """
  Get conversation from chat history
  """

  from langchain.schema import HumanMessage, AIMessage
  import langchain_core.messages

  for message in chat_history.messages:
      if type(message) is HumanMessage:
        print(f"Human : {message.content}")
      elif type(message) is AIMessage:
        print(f"AI : {message.content}")


def interactive_chat(query, session_id):
  """
  Chat with agent with chat history.
  """
  respone = agent.query(input=query,
                        config={"configurable": {"session_id": session_id}},
  )
  return respone['output']


### Unit test of Firestore

In [11]:
session_id = "conv-004"

chat_history = get_session_history(session_id)
chat_history.clear()
print(chat_history.messages)

[]


## Integrate reasoning engine and chat history

### Tools definition

In [12]:
# Define Google Search tool
from vertexai.generative_models import grounding, Tool
grounded_search_tool = Tool.from_google_search_retrieval(
    grounding.GoogleSearchRetrieval()
)

### Agent - Local unit test

In [14]:
from vertexai.preview.generative_models import ToolConfig

# Agent Define
agent = reasoning_engines.LangchainAgent(
    model=MODEL_NAME,
    tools=[grounded_search_tool,],
    chat_history=get_session_history,  # function to get FirestoreChatMessageHistory
    agent_executor_kwargs={"return_intermediate_steps": False},
)

### Add chat come from interaction into chat history


In [15]:
from time import perf_counter

while True:
  query = input('사용자: ')

  if query == '종료': break

  t1_start = perf_counter()
  output = interactive_chat(query, session_id)
  t1_end = perf_counter()

  display(Markdown(f"{t1_end-t1_start}"))
  display(Markdown(f"AI Agent : {output}"))
  print(f"------------------------------------ ")

chat_history = get_session_history(session_id)
chat_history.clear()
print(chat_history.messages)

사용자: 안녕하세요.


4.745462649999922

AI Agent : 안녕하세요 (annyeonghaseyo) is the most common greeting in Korean.  It literally translates to "are you at peace?" or "are you well?"  It's a polite and respectful way to greet someone, showing concern for their well-being.  The greeting encompasses more than a simple "hello"; it conveys a sense of wishing the other person good health and a peaceful state.  While often compared to "hello" or "good morning/evening" in English, it carries a deeper meaning reflecting Korean culture's emphasis on harmony and well-being.  The word is composed of the adjective 안녕하다 (annyeonghada, meaning peaceful or well) and honorific suffixes.  Its use is versatile, appropriate for various times of day and levels of formality.  Even in informal settings, it is generally preferred over less formal greetings.


------------------------------------ 
사용자: 최근 한국의 정치 상황이 어떻게 되나요 ?


4.076854176000097

AI Agent : 최근 한국의 정치 상황은 극심한 불안정에 놓여 있습니다.  한덕수 국무총리 권한대행에 대한 탄핵소추안이 야당에 의해 발의되었고,  이로 인해 정치적 혼란이 심화되고 경제에 부정적인 영향을 미치고 있습니다.

주요 외신들은 한국의 정치 위기를 심각하게 보도하고 있으며,  원화 가치 급락 등 경제적 어려움을 강조하고 있습니다.  특히,  미국 차기 대통령 트럼프의 보호무역 정책과 맞물려 한국 경제에 대한 우려가 더욱 커지고 있습니다.  

정치적 불확실성은 투자 심리를 위축시키고,  경제 회복을 저해하는 주요 요인으로 작용하고 있습니다.  여야 간의 정치적 대립이 심화되면서 국정 운영의 공백이 우려되고 있으며,  이러한 상황이 장기화될 경우, 한국 경제에 심각한 타격을 줄 수 있다는 분석이 지배적입니다.

미국 국무부는 한덕수 권한대행과 계속 협력할 의지를 표명했지만,  정치적 불안정이 미한일 협력에도 부정적 영향을 미칠 수 있다는 우려도 제기되고 있습니다.  현재 상황은  정치적 해결책을 찾지 못하면 경제적 위기로 이어질 가능성이 높은 매우 심각한 상태입니다.  이러한 상황은  오늘(2024년 12월 27일) 현재의 상황이며,  향후 변동될 가능성이 있습니다.


------------------------------------ 
사용자: 외신들이 왜 심각하게 생각하고 있죠 ?


6.156147646000022

AI Agent : 외신들이 한국의 정치 상황을 심각하게 보는 이유는 여러 가지가 복합적으로 작용하기 때문입니다.  요약하자면 다음과 같습니다.

* **정치적 불안정의 심화:**  잇따른 탄핵 시도는 한국 정치의 불안정성을 심각하게 드러냅니다.  한덕수 국무총리 권한대행에 대한 탄핵 소추안 발의는 단순한 정치적 갈등을 넘어, 정부의 기능 마비와 국정 공백에 대한 우려를 불러일으키고 있습니다.  이러한 정치적 혼란은 장기화될 가능성이 높고,  결국 정치 시스템에 대한 국제적 신뢰도를 떨어뜨릴 수 있습니다.

* **경제적 불확실성 증대:**  정치적 혼란은 경제에 직접적인 악영향을 미칩니다.  원화 가치 급락, 투자 심리 위축 등은 이미 현실로 나타나고 있으며, 외국인 투자 감소와 경제 성장 둔화로 이어질 수 있습니다. 특히,  트럼프 행정부의 보호무역주의 정책과 맞물려 한국 경제의 미래에 대한 불확실성이 더욱 커지고 있습니다.  이는 한국 경제의 안정성에 의문을 제기하며, 외국 투자자들의 신뢰를 잃게 만들 수 있습니다.

* **국제적 협력 저해:**  정치적 불안은  미국을 비롯한 주요국과의 협력에도 부정적 영향을 미칠 수 있습니다.  안정적인 정치 환경은 국제 사회에서의 신뢰 구축과 협력에 필수적인 요소입니다.  한국 정부의 불안정은 국제적인 협력 관계에 균열을 가져오고,  외교적 노력에 어려움을 야기할 수 있습니다.  미국과의 동맹 관계 유지에도 부정적 영향을 미칠 수 있습니다.

* **민주주의 시스템에 대한 우려:**  잦은 탄핵 시도는 한국 민주주의 시스템 자체에 대한 우려를 불러일으킵니다.  민주주의는 안정적인 정치 시스템을 전제로 하며,  빈번한 정권 교체나 정치적 혼란은 민주주의의 안정성을 해치고 국제 사회로부터 비판을 받을 수 있습니다.


결론적으로, 외신들이 한국의 정치 상황을 심각하게 받아들이는 것은 단순히 정치적 갈등을 넘어,  경제적 불확실성, 국제적 협력 저해, 민주주의 시스템에 대한 우려 등 다양한 요소들이 복합적으로 작용하기 때문입니다.  이러한 상황은 한국의 장기적인 안정과 발전에 심각한 위협이 될 수 있습니다.


------------------------------------ 
사용자: 종료
[]


In [18]:
session_id = "conv-003"
chat_history = get_session_history(session_id)
chat_history.messages

[HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={}),
 AIMessage(content='"안녕하세요"는 한국어에서 가장 일반적인 인사말입니다.  "안녕하세요"는 "무탈하시오?"와 같이 "잘 지내시나요?"라고 해석할 수 있습니다.  "안녕하세요"는 "안녕하다"에서 온 말인데, "안녕하다"는 "평화롭다" 또는 "편안하다"는 뜻입니다.  그래서 "안녕하세요"는 "잘 지내시나요?" 또는 "평안하신가요?"와 같은 의미를 지닙니다.\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='현재 한국의 경제 상황에 대해서 설명해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content=' 한국의 경제 상황은 현재 긍정적이지만, 일부 어려움을 겪고 있습니다. \n\n* **긍정적인 측면:**\n    * **수출:** 한국의 수출은 ICT 제품을 중심으로 양호한 흐름을 유지하고 있습니다. 특히 반도체 수출이 높은 증가세를 기록하고 있습니다. \n    * **경제 성장:** 2024년 한국 경제 성장률은 2.2%에서 2.7%로 상향 조정되었습니다. 이는 1분기 경제 성장률이 예상보다 높았고, 수출 경기 회복이 내수 부진을 일부 완화했기 때문입니다.\n    * **설비 투자:** 금리 인하와 반도체 경기 호조세로 설비 투자가 증가할 것으로 예상됩니다. 특히 반도체 관련 투자가 확대되고 있습니다.\n    * **경상수지:** 수출 증가세 둔화와 내수 회복에도 불구하고 교역 조건 개선으로 경상수지 흑자 폭이 확대될 전망입니다. \n    * **신용 등급:** 한국의 신용 등급은 안정적인 수준을 유지하고 있습니다.\n\n* **부정적인 측면:**\n    * **내수 부진:** 건설 투자가 부진하면서 내수 회복이 제약되고 있습니다. \n    * **고용:

In [19]:
# Manual input for chat history if you want to add something usefule to chat history.
chat_history.add_user_message("what's your role?")
chat_history.add_ai_message("I'm a helpful assistant")

chat_history.messages

[HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={}),
 AIMessage(content='"안녕하세요"는 한국어에서 가장 일반적인 인사말입니다.  "안녕하세요"는 "무탈하시오?"와 같이 "잘 지내시나요?"라고 해석할 수 있습니다.  "안녕하세요"는 "안녕하다"에서 온 말인데, "안녕하다"는 "평화롭다" 또는 "편안하다"는 뜻입니다.  그래서 "안녕하세요"는 "잘 지내시나요?" 또는 "평안하신가요?"와 같은 의미를 지닙니다.\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='현재 한국의 경제 상황에 대해서 설명해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content=' 한국의 경제 상황은 현재 긍정적이지만, 일부 어려움을 겪고 있습니다. \n\n* **긍정적인 측면:**\n    * **수출:** 한국의 수출은 ICT 제품을 중심으로 양호한 흐름을 유지하고 있습니다. 특히 반도체 수출이 높은 증가세를 기록하고 있습니다. \n    * **경제 성장:** 2024년 한국 경제 성장률은 2.2%에서 2.7%로 상향 조정되었습니다. 이는 1분기 경제 성장률이 예상보다 높았고, 수출 경기 회복이 내수 부진을 일부 완화했기 때문입니다.\n    * **설비 투자:** 금리 인하와 반도체 경기 호조세로 설비 투자가 증가할 것으로 예상됩니다. 특히 반도체 관련 투자가 확대되고 있습니다.\n    * **경상수지:** 수출 증가세 둔화와 내수 회복에도 불구하고 교역 조건 개선으로 경상수지 흑자 폭이 확대될 전망입니다. \n    * **신용 등급:** 한국의 신용 등급은 안정적인 수준을 유지하고 있습니다.\n\n* **부정적인 측면:**\n    * **내수 부진:** 건설 투자가 부진하면서 내수 회복이 제약되고 있습니다. \n    * **고용: